In [46]:
import pandas as pd
import os
import librosa
import numpy as np
import scipy
import re


#Pickling
from six.moves import cPickle as pickle

# Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import scale
#HTTP
import requests
import json

import matplotlib.pyplot as plt
import matplotlib.style as ms
import librosa.display
import IPython.display as ipd
import seaborn
ms.use('seaborn-muted')
%matplotlib inline

x_train = []
y_train = []
x_test = []
y_test = []
no_alignment_file = [4764]

In [47]:
def extract_patterns(data,extract=False):
    if(extract):
        patterns = {}
        for index, row in data.iterrows():
            patterns[row['index']] = set(get_pattern([row['text']])[0].values())
            print('Extracted pattern from '+ row['index'] + ' index:'+ str(index))
            print('Size: ', len(patterns[row['index']]), 'Patterns size', len(patterns))
        try:
            print('Saving Pickle')
            with open('pickles/patterns/pattern.pickle','wb') as f:
                save = {
                    'patterns' : patterns
                }
                pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
                print('Successfully saved in pattern.pickle')
                return patterns
        except Exception as e:
            print('Unable to save data to pickle', e)
            print('Patterns probably not saved.')
            return patterns
    else:
        try:
            with open('pickles/patterns/pattern.pickle','rb') as f:
                save = pickle.load(f)
                patterns = save['patterns']
                del save
                returning = {}
                for key in list(data['index']):
                    returning[key] = patterns[key]
                return returning
        except Exception as e:
            print('Error loading base datasets pickle: ', e)

def get_pattern(text):
    text = json.dumps(text)
    url = 'http://192.168.2.101:7878/api/get_patt'
    data = dict(input_tweets = text)
    resp = requests.post(url=url, data=data)
    r = json.loads(resp.text)
    return map(lambda x: x['pattern'],r)
    
def get_deep_emotion(text):
    text = json.dumps(text)
    url = 'http://192.168.2.101:7878/api/get_emo'
    data = dict(input_tweets = text)
    resp = requests.post(url=url, data=data)
    r = json.loads(resp.text)
    return r

def get_gender(text):
    text = text[len(text)-4]
    if('F' in text):
        return 'F'
    elif('M' in text):
        return 'M'
    else:
        return 'N/A'
    
def get_word_segments(align_file):
    segments = []
    try:
        with open(align_file,'r') as openFile:
            read_lines = openFile.readlines()
            lines = read_lines[1:len(read_lines)-1]
            for line in lines:
                splitted = line.split()
                segments.append({'sf': splitted[0], 'ef':splitted[1],'seg_scr':splitted[2],'word':splitted[3]})
    except Exception as e:
        print('Problem with file ',align_file)
    return segments

def clean_text(text):
    punct_str = '!"#$%&()*+,-./:;<=>?@\\^_`{|}~«»“…‘”\t'
    for p in punct_str:
        text = text.replace(p,' ')
    text = re.sub(' +', ' ', text)
    return text.lower().strip()

def play_audio(filename):
    y, sr = librosa.load(filename)
    return ipd.Audio(data=y,rate=sr)

# def play_audio(y,sr,start,end):
#     return ipd.Audio(data= y[start:end], rate=sr)
    
def extract_patterns(data,extract=False):
    if(extract):
        patterns = {}
        for index, row in data.iterrows():
            patterns[row['index']] = set(get_pattern([row['text']])[0].values())
            print('Extracted pattern from '+ row['index'] + ' index:'+ str(index))
            print('Size: ', len(patterns[row['index']]), 'Patterns size', len(patterns))
        try:
            print('Saving Pickle')
            with open('pickles/patterns/pattern.pickle','wb') as f:
                save = {
                    'patterns' : patterns
                }
                pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
                print('Successfully saved in pattern.pickle')
                return patterns
        except Exception as e:
            print('Unable to save data to pickle', e)
            print('Patterns probably not saved.')
            return patterns
    else:
        try:
            with open('pickles/patterns/pattern.pickle','rb') as f:
                save = pickle.load(f)
                patterns = save['patterns']
                del save
                returning = {}
                for key in list(data['index']):
                    returning[key] = patterns[key]
                return returning
        except Exception as e:
            print('Error loading base datasets pickle: ', e)
            
def filter_word_count(data, n_count):
    return data[list(map(lambda x: len(x.split(' ')) >= n_count,data['text']))]

def remove_empty_patterns(data,patterns):
    empty_patterns = [k for k, v in patterns.items() if len(v) < 1]
    patterns = { k:v for k, v in patterns.items() if len(v) > 1 }
    data = filter(lambda x: x[1]['index'] not in empty_patterns ,data.iterrows())
    data = pd.DataFrame.from_items(data).T
    return data,patterns

# Feature Extractors

In [48]:
def get_mfcc(y,sr,se_tup,n_mfcc=20):
    mfcc = librosa.feature.mfcc(y=y[se_tup[0]:se_tup[1]],sr=sr, n_mfcc=n_mfcc)
    mfcc = np.mean(mfcc,axis=1)
    return mfcc

def scale_features(x_train, x_test):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaled_x_train = scaler.fit_transform(x_train)
#     print(scaled_x_train.min(axis=0))
#     print(scaled_x_train.max(axis=0))
    scaled_x_test = scaler.fit_transform(x_test)
    return scaled_x_train,scaled_x_test

def get_rmse(y,se_tup):
    return np.sqrt(np.mean(y[se_tup[0]:se_tup[1]]**2))


def extract_features(y,sr, se_tup):
    results = [
        get_mfcc(y,sr,se_tup),
        get_rmse(y,se_tup),
    ]
    return results
    
                

In [49]:
def load_data(word_count,emotional_mapping):
    # full = generate_IEMOCAP_df()
    data = pd.read_csv('data/IEMOCAP_sentences.csv',index_col=0)
    data['emotion_code'] = data['emotion'].map( emotional_mapping ).astype(int)
    # Take away fear, surprise,disgust, xxx and others. Not enough data
    data = data[data.emotion_code < 6]
    #Remove rows that don't have Alignment file
    data = data.drop(no_alignment_file)
    # Clean Transcripts
    data['text'] = data['text'].apply(clean_text)
    # Filter Word Count
    data = filter_word_count(data, word_count)
    patterns = extract_patterns(data)
#     data,patterns = remove_empty_patterns(data,patterns)
    return data,patterns

def get_word_coordinate(word_segments,transcript, y_size):
    word_data = []
    start_data = []
    end_data = []
    silence = ['<sil>']
    actions = ['++garbage++','++breathing++', '++laughter++','++lipsmack++']
    if(len(word_segments) > 0 ):
        frames = map(lambda x: (x['sf'],x['ef']),word_segments)
        frame_size = int(frames[len(frames)-1][1])
        rate = y_size / frame_size
        word_segments.pop()
        word_segments.pop(0)
        text_tokens = transcript.split(' ')
        token_index = 0
        for wordsg in word_segments:
            word = wordsg['word'].lower().split('(')[0]
            if(word in silence):
                'There is a silence here'
            elif(word in actions):
                action = word
                token_index += 1
            else:
                word_data.append(word)
                start_data.append(int(wordsg['sf'])* rate)
                end_data.append(int(wordsg['ef'])* rate)
                
                if(word != text_tokens[token_index]):
                    print('ERROR')
                    print('Expected', text_tokens[token_index])
                    print('Found', word)
                    return None
                token_index +=1
        return [word_data,start_data,end_data]
    return None

def matchForcedAlign(patt,parsedData):
    wild_card = '.+'
    i = 0
    initWildCardCount = 0
    endWildCardCount = 0
    wordCount = 0
    word_index = []
    transcript = parsedData[0]
    passedWord = False
    pattern_tokens = patt.split(' ')
    for token in pattern_tokens:
        if(token == wild_card):
            if(passedWord):
                endWildCardCount += 1
                # TODO: Fix instance of [con_w wild_card con_w]
            else:
                initWildCardCount += 1
        else:
            passedWord = True
            wordCount += 1
            if word_index == []:
                word_index = [i for i, e in enumerate(transcript) if e == token]
    results = []
    for index in word_index:
        if((index - initWildCardCount) >= 0 and (index + wordCount - 1 + endWildCardCount) < len(transcript)):
            index_result = [(-1,-1),(-1,-1),(-1,-1)]
            if(initWildCardCount > 0):
                index_result[0] = (parsedData[1][index-initWildCardCount],parsedData[2][index - 1])
            if(endWildCardCount > 0):
                index_result[2] = (parsedData[1][index + wordCount],parsedData[2][index+ wordCount - 1 + endWildCardCount])
            index_result[1] = (parsedData[1][index],parsedData[2][index + wordCount - 1])
            results.append(index_result)
        else:
            word_index.remove(index)
    return word_index, results, [initWildCardCount,wordCount,endWildCardCount]


In [50]:
emotional_mapping = {'ang': 0, 'sad': 1, 'exc': 2, 'neu': 3,'fru': 4,'hap': 5,'fea': 6,'sur': 7,'dis': 8, 'xxx':9,'oth':10}
data,patterns = load_data(3,emotional_mapping)
data.groupby('emotion').count()['index']


emotion
ang    1017
exc     924
fru    1657
hap     523
neu    1418
sad     937
Name: index, dtype: int64

In [39]:
extractFeatures = False
feature_table = None
if(extractFeatures):
    wcards_b = False
    connectw_b = False
    all_patt = True
    feature_table = {}

    for pattern_name in patterns:
        row = data[data['index'] == pattern_name]
        emotion = row.emotion.values[0]
        wav_path = row.wav_path.values[0]
        align_path = row.alignment_path.values[0]
        segments = get_word_segments(align_path)
        transcript = row.text.values[0]
        y,sr = librosa.load(wav_path)
        parsedData = get_word_coordinate(segments,transcript,len(y))
        for patt in patterns[pattern_name]: 
            word_index, results, counts = matchForcedAlign(patt,parsedData)
            features_vect = []

            for result in results:
                if(all_patt):
                    x_temp = result[0][0]
                    y_temp = result[2][1]
                    if(x_temp == -1):
                        x_temp= result[1][0]
                    if(y_temp == -1):
                        y_temp = result[1][1]

                    all_patt_feature = extract_features(y,sr,(x_temp,y_temp))
                    features_vect.append(all_patt_feature)
                if(wcards_b):
                    if(result[0][0] != -1 and result[2][0] != -1):
                        temp_y = np.concatenate([y[result[0][0]:result[0][1]] , y[result[2][0]:result[2][1]]])
                        wc_feat = extract_features(temp_y,sr,(0,len(temp_y)-1))
                        features_vect.append(wc_feat)
                    else:
                        if(result[0][0] != -1):
                            i_wc_i = result[0]
                            i_wc_feat = extract_features(y,sr,i_wc_i)
                            features_vect.append(i_wc_feat)
                        if(result[2][0] != -1):
                            e_wc_i = result[2]
                            e_wc_feat = extract_features(y,sr,e_wc_i)
                            features_vect.append(e_wc_feat)
                if(connectw_b):
                    cw_i = result[1]
                    cw_feat = extract_features(y,sr,cw_i)
                    features_vect.append(cw_feat)
                break
            if(len(results) > 1):
                print('MORE THAN ONE',results)
                #TODO: Just considering the first find in the Pattern Detected 
            print('Extracted feature from pattern',patt,'in recording',pattern_name)
            feature_table[(pattern_name + '<PATTERN>'+ patt)] = features_vect
    try:
        print('Saving Pickle')
        with open('pickles/patterns/pattern_features.pickle','wb') as f:
            save = {
                'feature_table' : feature_table
            }
            pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
            print('Successfully saved in pattern_features.pickle')
    except Exception as e:
        print('Unable to save data to pickle', e)
else:
    try:
        with open('pickles/patterns/pattern_features.pickle','rb') as f:
            save = pickle.load(f)
            feature_table = save['feature_table']
            del save
    except Exception as e:
        print('Error loading pattern features pickle: ', e)



Error loading pattern features pickle:  'ascii' codec can't decode byte 0xe4 in position 0: ordinal not in range(128)


In [44]:
save = pd.read_pickle('pickles/patterns/pattern_features.pickle')
feature_table = save['feature_table']
del save

In [45]:
feature_table

{'Ses03M_impro08a_M019<PATTERN>.+ through': [[array([-4.81830820e+02,  1.59664409e+02, -1.80356637e+01,  7.09126990e+01,
          -4.05491026e-01,  2.12183166e+01, -5.26466949e+00, -3.26901914e+00,
           5.91250951e-01, -1.66887489e+01, -2.52675862e+00, -1.28656044e+01,
           5.81393301e+00, -1.25869640e+01, -4.65971371e+00,  4.40350179e+00,
          -1.48914836e+01, -3.94845510e+00, -3.99964658e-01, -1.38361262e+00]),
   0.010872793]],
 'Ses02M_impro07_M017<PATTERN>the .+ .+': [[array([-2.95594206e+02,  1.65361324e+02, -4.33112603e+01,  7.14592890e+01,
          -1.45175390e+01, -3.85889736e+00, -9.32373575e+00, -3.37130092e+00,
          -1.08837630e+01, -1.38149507e+01,  6.82918644e+00, -1.76816099e+01,
           1.70187227e+00, -1.23568934e+01,  2.86444252e-01,  6.59902428e+00,
          -1.40820941e+01,  4.23201859e-01, -7.98146460e+00, -4.76985522e+00]),
   0.0491465]],
 'Ses01M_impro03_F010<PATTERN>just .+': [[array([-196.27052816,   81.95566471,  -69.20609162,   82